In [1]:
import pandas as pd
from pathlib import Path
import toml
from typing import List
from dataclasses import dataclass, asdict
from IPython.display import Markdown

%run _utils.py

def get_jb_files(fdir=Path('.')):
    exts = ['*.ipynb', '*.md']
    fnms = []
    for e in exts: 
        tmp = recursive_glob(rootdir=fdir, pattern=e, recursive=True)
        fnms.extend(tmp)
    fpths = [Path(fnm) for fnm in fnms if ".ipynb_checkpoints" not in fnm]
    names = [p.stem for p in fpths]
    return dict(zip(names,fpths))

def create_img_fpth(tech):
    """creates fpth from name"""
    for t in tech['tech']:
        if t['image'] == '':
            name = t['name']
            t['image'] = f'images/{name}-icon.png'

def create_local_link(tech, paths_map):
    """"""
    for t in tech['tech']:
        if t['local_link'] == '':
            for tag in t['tags']:
                if tag in paths_map.keys():
                    t['local_link'] = paths_map[tag]

@dataclass
class MfTech:
    name: str
    url: str
    description: str
    local_link: str
    image: str

@dataclass
class TechGroup:
    tag: str
    description: str
    local_link: str
    tech: List[MfTech] 


def create_tech_group(df, tag, tag_des) -> TechGroup:

    li_tmp = []
    df_tmp = srch_tgs(df, 'tags', tag, clmn_rtrn=list(df)).reset_index()
    keys = list(MfTech.__dict__['__annotations__'].keys())
    for index, row in df_tmp.iterrows():
        di = { key: dict(row)[key] for key in keys }
        tech = MfTech(**di)
        li_tmp.append(tech)
    if tag in tag_des.keys():
        description = tag_des[tag]
    else:
        description = ""
    local_link = df_tmp.local_link.unique()[0]
    return TechGroup(tag=tag, tech=li_tmp, description=description, local_link=local_link)

def create_tech_groups(df, tags, tag_des):

    li_tech_groups = []
    for tag in tags:
        li_tech_groups.append(create_tech_group(df, tag, tag_des))
    return tuple(li_tech_groups)


def md_img_link(tech: MfTech)-> str:
    return f"[![{tech.name}]({tech.image})]({tech.url} '{tech.description}')"

def md_img_links(grp: TechGroup)-> str:
    return "\n".join([md_img_link(tech) for tech in grp.tech])

def create_panel_obj(grp: TechGroup) -> str:
    imgs = md_img_links(grp)
    return f"""
**[{grp.tag}]({grp.local_link})**
^^^
{grp.description}

{imgs}
---
"""
def create_panel_objs(li_grps: List[TechGroup]) -> str:
    return "\n".join([create_panel_obj(grp) for grp in li_grps])

def create_panel(tu_tech_grps: List[TechGroup]) -> str:
    output = """

:::{panels}
:container: +full-width text-center
:column: col-lg-4 px-2 py-2
:card:

""" + create_panel_objs(tu_tech_grps) + "\n :::"
    
    return output

# mfcode_docs

**Max Fordham Engineering Development Best Practice**

This documentation aims to provide high-level, non-verbose notes outlining Max Fordham's engineering software development infrastructure.
Links to external sources and standards to describe the approach are preferred where possible.
The intended audience are those contributing-to and maintaining internal development tools, or external parties and collaborators interested in our approach.
Code snippets are encouraged.

**We stand on the shoulders of giants**. Our software is built on robust, high-quality opensource packages, and we are proud to share the work that we produce with the opensource community.
When writing code there are always many solutions to a problem;
we standardise the technologies we use for a given task to ensure a consistent approach across projects. See below for key technologies on which we rely.

In [4]:
path_tech = Path('_preferred_technologies.toml')
path_docs = Path('.')

tech = toml.load(path_tech)
create_img_fpth(tech)
paths_map = get_jb_files(path_docs)
create_local_link(tech, paths_map)

df = pd.DataFrame.from_dict(tech['tech'])
tags = list(set(flatten_list(df.tags.tolist())))
tag_des = tech['tags']

tu_tech_grps = create_tech_groups(df, tags, tag_des)
md = create_panel(tu_tech_grps)

from IPython.display import display

display(Markdown(md))



:::{panels}
:container: +full-width text-center
:column: col-lg-4 px-2 py-2
:card:


**[object-models](developing.md)**
^^^


[![pydantic](images/pydantic-icon.png)](https://pydantic-docs.helpmanual.io/ 'Define how data should be in pure, canonical python; validate it with pydantic.
Used to create object definitions that feed: databases, calculation tools and documentation. 
As Linus said:
"Bad programmers worry about the code. Good programmers worry about data structures and their relationships."
with a good object model, the code becomes easy.
')
---


**[external-experts](developing.md)**
^^^


[![quantstack](images/quantstack-icon.png)](https://quantstack.net/ 'for help with the heavy lifting and deployment we call in the pros.
Quantstack and one of the core Jupyter developers and are very active members of the opensource community. 
working with them means we can feel confident in our approach to software development, whilst concentrating on what we're best at: Engineering.
')
---


**[backend](databases.md)**
^^^


[![postgresql](images/postgresql-icon.png)](https://www.postgresql.org/ 'PostgreSQL is a powerful, open source object-relational database system
with over 30 years of active development that has earned it a strong reputation for reliability, feature robustness, and performance.
It is used for production-grade tools.
')
[![sqlite](images/sqlite-icon.png)](https://www.sqlite.org/index.html 'SQLite is a C-language library that implements a small, fast, self-contained, high-reliability, full-featured, SQL database engine.
It is used for some standalone apps and prototyping.
')
[![sqlalchemy](images/sqlalchemy-icon.png)](https://www.sqlalchemy.org/ 'SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.
Table definitions and relations are developed in SQL alchemy, as well as data-serialisation with pydantic.
')
[![pydantic](images/pydantic-icon.png)](https://pydantic-docs.helpmanual.io/ 'Define how data should be in pure, canonical python; validate it with pydantic.
Used to create object definitions that feed: databases, calculation tools and documentation. 
As Linus said:
"Bad programmers worry about the code. Good programmers worry about data structures and their relationships."
with a good object model, the code becomes easy.
')
---


**[developing](software-automation.md)**
^^^


[![pyrevit](images/pyrevit-icon.png)](https://www.notion.so/pyrevitlabs/pyRevit-bd907d6292ed4ce997c46e84b6ef67a0 'pyRevit (with lowercase py) is a Rapid Application Prototyping (RAD) environment for Autodesk Revit.
It helps you quickly sketch out your automation and add-on ideas, in whichever language that you are most comfortable with, 
inside the Revit environment and using its APIs. 
')
[![grasshopper+ladybug-tools](images/grasshopper+ladybug-tools-icon.png)](https://www.food4rhino.com/en/app/ladybug-tools 'visual coding and python scripting environment inside McNeel Rhino.
good for early-stage concept design development and analysis.
')
[![IES-vescripts](images/IES-vescripts-icon.png)](https://www.iesve.com/software/python-scripting 'You can use the VE Python API to create your own customised scripts to automate reports, data input, parametric simulations and many other tasks.
')
[![voila](images/voila-icon.png)](https://www.iesve.com/software/python-scripting 'Voilà turns Jupyter notebooks into standalone web applications.')
[![postgresql](images/postgresql-icon.png)](https://www.postgresql.org/ 'PostgreSQL is a powerful, open source object-relational database system
with over 30 years of active development that has earned it a strong reputation for reliability, feature robustness, and performance.
It is used for production-grade tools.
')
[![sqlite](images/sqlite-icon.png)](https://www.sqlite.org/index.html 'SQLite is a C-language library that implements a small, fast, self-contained, high-reliability, full-featured, SQL database engine.
It is used for some standalone apps and prototyping.
')
[![sqlalchemy](images/sqlalchemy-icon.png)](https://www.sqlalchemy.org/ 'SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.
Table definitions and relations are developed in SQL alchemy, as well as data-serialisation with pydantic.
')
[![pydantic](images/pydantic-icon.png)](https://pydantic-docs.helpmanual.io/ 'Define how data should be in pure, canonical python; validate it with pydantic.
Used to create object definitions that feed: databases, calculation tools and documentation. 
As Linus said:
"Bad programmers worry about the code. Good programmers worry about data structures and their relationships."
with a good object model, the code becomes easy.
')
[![quantstack](images/quantstack-icon.png)](https://quantstack.net/ 'for help with the heavy lifting and deployment we call in the pros.
Quantstack and one of the core Jupyter developers and are very active members of the opensource community. 
working with them means we can feel confident in our approach to software development, whilst concentrating on what we're best at: Engineering.
')
---


**[deploying](deploying.md)**
^^^
a server hosted jupyterhub gives all Max Fordham users access to jupyter notebooks and apps.
the server is setup, deployed and updated using ansible.
the jupyterhub repo2docker plugin creates docker images of apps and notebook environments.

see [jupyterhub-maxfordham](https://github.com/gunstonej/jupyterhub-maxfordham) for MF setup and configuration.
the work was done by quantstack, and copies a previous set-up they configured [plasmabio](https://docs.plasmabio.org/en/latest/overview/index.html).


[![ansible](images/ansible-icon.png)](https://docs.ansible.com/ 'Ansible playbooks are used to automate the setup, configuration and update
of remote servers from a local machine. This is used to deploy our JupyterHub server.')
[![jupyterhub](images/jupyterhub-icon.png)](https://jupyterhub.readthedocs.io/en/stable/ 'JupyterHub is the best way to serve Jupyter notebook for multiple users.
It can be used in a class of students, a corporate data science group or scientific research group. 
')
[![repo2docker](images/repo2docker-icon.png)](https://repo2docker.readthedocs.io/en/latest/index.html 'repo2docker fetches a repository (from GitHub, GitLab, Zenodo, Figshare, Dataverse installations, a Git repository or a local directory)
and builds a container image in which the code can be executed. 
The image build process is based on the configuration files found in the repository.
')
---


**[frontend](developing.md)**
^^^


[![voila](images/voila-icon.png)](https://www.iesve.com/software/python-scripting 'Voilà turns Jupyter notebooks into standalone web applications.')
---


**[software-automation](software-automation.md)**
^^^
some of our core engineering design and analysis software (namely: Revit, Rhino and IES) have open APIs and python scripting
functionality. Scripts and custom interface elements are developed to automate tasks, or coordinate data with the larger
Max Fordham development ecosystem.


[![pyrevit](images/pyrevit-icon.png)](https://www.notion.so/pyrevitlabs/pyRevit-bd907d6292ed4ce997c46e84b6ef67a0 'pyRevit (with lowercase py) is a Rapid Application Prototyping (RAD) environment for Autodesk Revit.
It helps you quickly sketch out your automation and add-on ideas, in whichever language that you are most comfortable with, 
inside the Revit environment and using its APIs. 
')
[![grasshopper+ladybug-tools](images/grasshopper+ladybug-tools-icon.png)](https://www.food4rhino.com/en/app/ladybug-tools 'visual coding and python scripting environment inside McNeel Rhino.
good for early-stage concept design development and analysis.
')
[![IES-vescripts](images/IES-vescripts-icon.png)](https://www.iesve.com/software/python-scripting 'You can use the VE Python API to create your own customised scripts to automate reports, data input, parametric simulations and many other tasks.
')
---


**[databases](databases.md)**
^^^
insight can be extract from structured data. our automation tools create structured data that is passed to
databases for global review and analysis.


[![postgresql](images/postgresql-icon.png)](https://www.postgresql.org/ 'PostgreSQL is a powerful, open source object-relational database system
with over 30 years of active development that has earned it a strong reputation for reliability, feature robustness, and performance.
It is used for production-grade tools.
')
[![sqlite](images/sqlite-icon.png)](https://www.sqlite.org/index.html 'SQLite is a C-language library that implements a small, fast, self-contained, high-reliability, full-featured, SQL database engine.
It is used for some standalone apps and prototyping.
')
---


**[orm](developing.md)**
^^^


[![sqlalchemy](images/sqlalchemy-icon.png)](https://www.sqlalchemy.org/ 'SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.
Table definitions and relations are developed in SQL alchemy, as well as data-serialisation with pydantic.
')
---


**[packaging](packaging.md)**
^^^
The first newline is
trimmed in raw strings.
All other whitespace
is preserved.


[![conda-forge](images/conda-forge-icon.png)](https://conda-forge.org/ 'community maintained collection of opensource packages and template specifcation for new packages.')
[![conda-build](images/conda-build-icon.png)](https://docs.conda.io/en/latest/conda-build.html 'tool for building conda packages, maintained by anaconda.')
[![mamba](images/mamba-icon.png)](https://docs.conda.io/en/latest/conda-build.html 'Mamba is a fast, robust, and cross-platform package manager; it is a reimplementation of the conda package manager in C++.')
---

 :::